In [ ]:
import os
import sys
import pickle
import numpy as np
import pandas as pd
import glob
from scipy import stats
import cv2
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# from calib.utils import load_scene
from lib import utils, calib, plotting
from lib.points import get_pairwise_3d_points_from_df
from lib.app import reconstruction_reprojection_video, combine_dlc_vids
from lib.misc import rot_x, rot_y, rot_z, get_markers

ROOT_DATA_DIR = os.path.join("..", "data")

# Reconstruction Params.
You **must** define these - do not delete.<br>
Thereafter, run all cells

In [ ]:
# DATA_DIR = os.path.join(ROOT_DATA_DIR, "2019_03_09", "lily", "run")
DATA_DIR = os.path.join(ROOT_DATA_DIR, "2017_08_29 - Copy", "top", "jules", "run1_1")

start_frame = 20
end_frame = 70

# DLC p_cutoff - any points with likelihood < dlc_thresh are not trusted in EKF
dlc_thresh = 0.8  # change this only if EKF result is unsatisfactory

include_lure = True

# EKF

## Init vars

In [ ]:
# Define the indices for the states
state_id = [
    'x_0', 'y_0', 'z_0',         # head position in inertial
    'phi_0', 'theta_0', 'psi_0', # head rotation in inertial
    'phi_1', 'theta_1', 'psi_1', # neck
    'theta_2',                   # front torso
    'phi_3', 'theta_3', 'psi_3', # back torso
    'theta_4', 'psi_4',          # tail_base
    'theta_5', 'psi_5',          # tail_mid
    'theta_6', 'theta_7',        # l_shoulder, l_front_knee
    'theta_8', 'theta_9',        # r_shoulder, r_front_knee
    'theta_10', 'theta_11',      # l_hip, l_back_knee
    'theta_12', 'theta_13',      # r_hip, r_back_knee
    ]

# define DLC labels
markers = get_markers(include_lure)

if include_lure:
    state_id += ['x_l', 'y_l', 'z_l'] # lure position in inertial

n_states = 3*len(state_id)
vel_idx = n_states//3
acc_idx = n_states*2//3
n_markers = len(markers)

idx = {}
for i, deriv_idx in enumerate([0, vel_idx, acc_idx]):
    idx.update({'d'*i+state: deriv_idx+j for j, state in enumerate(state_id)})

# Load extrinsic params
k_arr, d_arr, r_arr, t_arr, cam_res, n_cams, scene_fpath = utils.find_scene_file(DATA_DIR)
camera_params = [[K, D, R, T] for K, D, R, T in zip(k_arr, d_arr, r_arr, t_arr)]

# other vars
start_frame -= 1 # 0 based indexing
n_frames = end_frame - start_frame
sigma_bound = 3
max_pixel_err = cam_res[0] # used in measurement covariance R
sT = 1/90 if max_pixel_err == 1920 else 1/120 # timestep

## Function definitions

In [ ]:
def get_3d_marker_coords(x: np.ndarray):
    """Returns a numpy array of the 3D marker coordinates (shape Nx3) for a given state vector x.
    """
    # 1. Rotations
    RI_0 = rot_z(x[idx['psi_0']]) @ rot_x(x[idx['phi_0']]) @ rot_y(x[idx['theta_0']]) # Head
    R0_I = RI_0.T
    RI_1 = rot_z(x[idx['psi_1']]) @ rot_x(x[idx['phi_1']]) @ rot_y(x[idx['theta_1']]) @ RI_0 # Neck
    R1_I = RI_1.T
    RI_2 = rot_y(x[idx['theta_2']]) @ RI_1 # Front torso
    R2_I = RI_2.T
    RI_3 = rot_z(x[idx['psi_3']]) @ rot_x(x[idx['phi_3']]) @ rot_y(x[idx['theta_3']]) @ RI_2 # Back Torso
    R3_I = RI_3.T
    RI_4 = rot_z(x[idx['psi_4']]) @ rot_y(x[idx['theta_4']]) @ RI_3 # Tail base
    R4_I = RI_4.T
    RI_5 = rot_z(x[idx['psi_5']]) @ rot_y(x[idx['theta_5']]) @ RI_4 # Tail mid
    R5_I = RI_5.T
    RI_6 = rot_y(x[idx['theta_6']]) @ RI_2 # l_shoulder
    R6_I = RI_6.T
    RI_7 = rot_y(x[idx['theta_7']]) @ RI_6 # l_front_knee
    R7_I = RI_7.T
    RI_8 = rot_y(x[idx['theta_8']]) @ RI_2 # r_shoulder
    R8_I = RI_8.T
    RI_9 = rot_y(x[idx['theta_9']]) @ RI_8 # r_front_knee
    R9_I = RI_9.T
    RI_10 = rot_y(x[idx['theta_10']]) @ RI_3 # l_hip
    R10_I = RI_10.T
    RI_11 = rot_y(x[idx['theta_11']]) @ RI_10 # l_back_knee
    R11_I = RI_11.T
    RI_12 = rot_y(x[idx['theta_12']]) @ RI_3 # r_hip
    R12_I = RI_12.T
    RI_13 = rot_y(x[idx['theta_13']]) @ RI_12 # r_back_knee
    R13_I = RI_13.T

    # 2. Positions
    p_head = np.array([x[[idx['x_0'], idx['y_0'], idx['z_0']]]]).T
    p_l_eye = p_head + R0_I @ np.array([[0, 0.03, 0]]).T
    p_r_eye = p_head + R0_I @ np.array([[0, -0.03, 0]]).T
    p_nose = p_head + R0_I @ np.array([[0.055, 0, -0.055]]).T
    
    p_neck_base = p_head + R1_I @ np.array([[-0.28, 0, 0]]).T
    p_spine = p_neck_base + R2_I @ np.array([[-0.37, 0, 0]]).T
    
    p_tail_base = p_spine + R3_I @ np.array([[-0.37, 0, 0]]).T
    p_tail_mid = p_tail_base + R4_I @ np.array([[-0.28, 0, 0]]).T
    p_tail_tip = p_tail_mid + R5_I @ np.array([[-0.36, 0, 0]]).T
    
    p_l_shoulder = p_neck_base + R2_I @ np.array([[-0.04, 0.08, -0.10]]).T
    p_l_front_knee = p_l_shoulder + R6_I @ np.array([[0, 0, -0.24]]).T
    p_l_front_ankle = p_l_front_knee + R7_I @ np.array([[0, 0, -0.28]]).T
    
    p_r_shoulder = p_neck_base + R2_I @ np.array([[-0.04, -0.08, -0.10]]).T
    p_r_front_knee = p_r_shoulder + R8_I @ np.array([[0, 0, -0.24]]).T
    p_r_front_ankle = p_r_front_knee + R9_I @ np.array([[0, 0, -0.28]]).T

    p_l_hip = p_tail_base + R3_I @ np.array([[0.12, 0.08, -0.06]]).T
    p_l_back_knee = p_l_hip + R10_I @ np.array([[0, 0, -0.32]]).T
    p_l_back_ankle = p_l_back_knee + R11_I @ np.array([[0, 0, -0.25]]).T

    p_r_hip = p_tail_base + R3_I @ np.array([[0.12, -0.08, -0.06]]).T
    p_r_back_knee = p_r_hip + R12_I @ np.array([[0, 0, -0.32]]).T
    p_r_back_ankle = p_r_back_knee + R13_I @ np.array([[0, 0, -0.25]]).T

    coords_3d = np.array([
        p_l_eye, p_r_eye, p_nose,
        p_neck_base, p_spine, 
        p_tail_base, p_tail_mid, p_tail_tip,
        p_l_shoulder, p_l_front_knee, p_l_front_ankle,
        p_r_shoulder, p_r_front_knee, p_r_front_ankle,
        p_l_hip, p_l_back_knee, p_l_back_ankle,
        p_r_hip, p_r_back_knee, p_r_back_ankle,
        ])

    if include_lure:
        p_lure = np.array([x[[idx['x_l'], idx['y_l'], idx['z_l']]]]).T
        coords_3d = np.concatenate([coords_3d, [p_lure]])
    
    return np.reshape(coords_3d, (len(coords_3d), 3))


def h_function(x: np.ndarray, k: np.ndarray, d: np.ndarray, r: np.ndarray, t: np.ndarray):
    """Returns a numpy array of the 2D marker pixel coordinates (shape Nx2) for a given state vector x and camera parameters k, d, r, t.
    """
    coords_3d = get_3d_marker_coords(x)
    coords_2d = calib.project_points_fisheye(coords_3d, k, d, r, t) # Project the 3D positions to 2D
    
    return coords_2d


def predict_next_state(x: np.ndarray, dt: np.float32):
    """Returns a numpy array of the predicted states for a given state vector x and time delta dt.
    """
    acc_prediction = x[acc_idx:]
    vel_prediction = x[vel_idx:acc_idx] + dt*acc_prediction
    pos_prediction = x[:vel_idx] + dt*vel_prediction + (0.5*dt**2)*acc_prediction
    
    return np.concatenate([pos_prediction, vel_prediction, acc_prediction]).astype(np.float32)


def numerical_jacobian(func, x: np.ndarray, *args):
    """Returns a numerically approximated jacobian of func with respect to x.
    Additional parameters will be passed to func using *args in the format: func(*x, *args)
    """
    n = len(x)
    eps = 1e-3
    
    fx = func(x, *args).flatten()
    xpeturb=x.copy()
    jac = np.empty((len(fx), n))
    for i in range(n):
        xpeturb[i] = xpeturb[i]+eps
        jac[:,i] = (func(xpeturb, *args).flatten() - fx)/eps
        xpeturb[i]=x[i]
        
    return jac

def save_ekf(states, smooth_states, path, start_frame, lure_included, dlc_thresh):
    import scipy.io
    types = ["", "_padded"]
    lure_str = "_with_lure" if lure_included else ""
    ekf_path = os.path.join(path, "ekf")
    os.makedirs(ekf_path, exist_ok=True)
    
    for t in types:
        
        if "padded" in t:
            nan_arr = np.full((start_frame, states.shape[1]), np.nan)

            states = np.concatenate((nan_arr, states))
            smooth_states = np.concatenate((nan_arr, smooth_states))
        
        positions = np.array([get_3d_marker_coords(state) for state in states])
        smooth_positions = np.array([get_3d_marker_coords(state) for state in smooth_states])
        
        file_data = dict(
            ekf_positions=positions,
            smooth_positions=smooth_positions,
            ekf_states=states,
            smooth_states=smooth_states,
        )

        with open(os.path.join(ekf_path, f"ekf{lure_str}{t}.pickle"), 'wb') as f:
            pickle.dump(file_data, f)
        
        scipy.io.savemat(os.path.join(ekf_path, f'ekf{lure_str}{t}.mat'), file_data)
        
        if "padded" in t:
            out_fpath = os.path.join(ekf_path, "ekf.avi")
            reconstruction_reprojection_video(path, out_fpath, smooth_positions, include_lure=lure_included, dlc_thresh=dlc_thresh)
        
    print('\nSaved files to', ekf_path)

## Load DLC data

In [ ]:
# Load DLC 2D point files (.h5 outputs)
dlc_2d_point_files = sorted(glob.glob(os.path.join(DATA_DIR, '*.h5')))
assert(len(dlc_2d_point_files) == n_cams), f"# of dlc '.h5' files != # of cams in {n_cams}_cam_scene_sba.json"

# Load Measurement Data (pixels, likelihood)
points_2d_df = utils.load_dlc_points_as_df(dlc_2d_point_files)
print("DLC points dataframe:\n", points_2d_df)

# Restructure dataframe
points_df = points_2d_df.set_index(['frame', 'camera','marker'])
points_df = points_df.stack().unstack(level=1).unstack(level=1).unstack()

# Pixels array
pixels_df = points_df.loc[:, (range(n_cams), markers, ['x','y'])]
pixels_df = pixels_df.reindex(columns=pd.MultiIndex.from_product([range(n_cams), markers, ['x','y']]))
pixels_arr = pixels_df.to_numpy() #shape - (n_frames, n_cams * n_markers * 2)

# Likelihood array
likelihood_df = points_df.loc[:, (range(n_cams), markers, 'likelihood')]
likelihood_df = likelihood_df.reindex(columns=pd.MultiIndex.from_product([range(n_cams), markers, ['likelihood']]))
likelihood_arr = likelihood_df.to_numpy() #shape - (n_frames, n_cams * n_markers * 1)

## Initialise EKF matrices

In [ ]:
points_3d_df = get_pairwise_3d_points_from_df(
    points_2d_df[points_2d_df['likelihood']>dlc_thresh], # ignore points with low likelihood
    k_arr, d_arr.reshape((-1,4)), r_arr, t_arr,
    calib.triangulate_points_fisheye
)

# estimate initial points
nose_pts = points_3d_df[points_3d_df["marker"]=="nose"][["frame", "x", "y", "z"]].values
nose_x_slope, nose_x_intercept, *_ = stats.linregress(nose_pts[:,0], nose_pts[:,1]) 
nose_y_slope, nose_y_intercept, *_ = stats.linregress(nose_pts[:,0], nose_pts[:,2])

nose_x_est = start_frame*nose_x_slope + nose_x_intercept # initial nose x
nose_y_est = start_frame*nose_y_slope + nose_y_intercept # initial nose y
nose_psi_est = np.arctan2(nose_y_slope, nose_x_slope)    # initial yaw angle relative to inertial

# INITIAL STATES
states = np.zeros(n_states)
states[[idx['x_0'], idx['y_0'],idx['psi_0']]] = [nose_x_est, nose_y_est, nose_psi_est] # head x, y & psi (yaw) in inertial
states[[idx['dx_0'], idx['dy_0']]] = [nose_x_slope/sT, nose_y_slope/sT]                # head x & y velocity in inertial

# INITIAL STATE COVARIANCE P - how much do we trust the initial states
# acceleration
p_lin_acc = np.ones(3)*3**2
p_ang_acc = np.ones(22)*3**2
p_ang_acc[10:] = 5**2
# velocity
p_lin_vel = np.ones(3)*5**2          # Know this within 2.5m/s and it's a uniform random variable
p_ang_vel = np.ones(22)*3**2
# position
p_lin_pos = np.ones(3)*3**2          # Know initial position within 4m
p_ang_pos = np.ones(22)*(np.pi/4)**2 # Know initial angles within 60 degrees, heading may need to change

P_list = [p_lin_pos, p_ang_pos, p_lin_vel, p_ang_vel, p_lin_acc, p_ang_acc]

# PROCESS COVARIANCE Q - how "noisy" the constant acceleration model is
qb_list = [
    5.0, 5.0, 5.0,    # head x, y, z in inertial
    10.0, 10.0, 10.0, # head phi, theta, psi in inertial
    5.0, 25.0, 5.0,   # neck phi, theta, psi
    50.0,             # front-torso theta
    5.0, 50.0, 25.0,  # back torso phi, theta, psi
    100.0, 30.0,      # tail base theta, psi
    140.0, 40.0,      # tail mid theta, psi
    350.0, 200.0,     # l_shoulder theta, l_front_knee theta
    350.0, 200.0,     # r_shoulder theta, r_front_knee theta
    450.0, 400.0,     # l_hip theta, l_back_knee theta
    450.0, 400.0,     # r_hip theta, r_back_knee theta
]

if include_lure:
    lure_pts = points_3d_df[points_3d_df["marker"]=="lure"][["frame", "x", "y", "z"]].values
    lure_x_slope, lure_x_intercept, *_ = stats.linregress(lure_pts[:,0], lure_pts[:,1]) 
    lure_y_slope, lure_y_intercept, *_ = stats.linregress(lure_pts[:,0], lure_pts[:,2])
    
    lure_x_est = start_frame*lure_x_slope + lure_x_intercept # initial lure x
    lure_y_est = start_frame*lure_y_slope + lure_y_intercept # initial lure y
    
    states[[idx['x_l'], idx['y_l']]] = [lure_x_est, lure_y_est]             # lure x & y in inertial
    states[[idx['dx_l'], idx['dy_l']]] = [lure_x_slope/sT, lure_y_slope/sT] # lure x & y velocity in inertial
    
    p_lure_acc = np.ones(3)*3**2
    p_lure_vel = np.ones(3)*5**2 # Know this within 2.5m/s and it's a uniform random variable
    p_lure_pos = np.ones(3)*3**2 # Know initial position within 4m
    
    P_list.insert(2, P_list[0]) # lure pos initial state covariance same as head
    P_list.insert(5, P_list[2]) # lure vel initial state covariance same as head
    P_list.append(P_list[6])    # lure accel initial state covariance same as head
    
    qb_list += qb_list[0:3] # lure x, y, z in inertial - same as head

P = np.diag(np.concatenate(P_list))

qb = (np.diag(qb_list)/2)**2
Q = np.block([
    [sT**4/4 * qb, sT**3/2 * qb, sT**2/2 * qb],
    [sT**3/2 * qb, sT**2 * qb, sT * qb],
    [sT**2/2 * qb, sT * qb, qb],
])

# MEASUREMENT COVARIANCE R
dlc_cov = 5**2

# State prediction function jacobian F - shape: (n_states, n_states)
rng = np.arange(n_states - vel_idx)
rng_acc = np.arange(n_states - acc_idx)
F = np.eye(n_states)
F[rng, rng+vel_idx] = sT
F[rng_acc, rng_acc+acc_idx] = sT**2/2

## Run EKF & Smoother

In [ ]:
# Allocate space for storing EKF data
states_est_hist = np.zeros((n_frames, n_states))
states_pred_hist = states_est_hist.copy()
P_est_hist = np.zeros((n_frames, n_states, n_states))
P_pred_hist = P_est_hist.copy()

outliers_ignored = 0

for i in range(n_frames):
    print(f"Running frame {i+start_frame+1}\r", end='')
    
    # ========== PREDICTION ==========

    # Predict State
    states = predict_next_state(states, sT).flatten()
    states_pred_hist[i] = states

    # Projection of the state covariance
    P = F @ P @ F.T + Q
    P_pred_hist[i] = P
    
    # ============ UPDATE ============
    
    z_k = pixels_arr[i+start_frame]
    likelihood = likelihood_arr[i+start_frame]
    
    # Measurement
    H = np.zeros((n_cams*n_markers*2, n_states))
    h = np.zeros((n_cams*n_markers*2)) # same as H[:, 0].copy()
    for j in range(n_cams):
        # State measurement
        h[j*n_markers*2:(j+1)*n_markers*2] = h_function(states[:vel_idx], *camera_params[j]).flatten()
        # Jacobian - shape: (2*n_markers, n_states)
        H[j*n_markers*2:(j+1)*n_markers*2, 0:vel_idx] = numerical_jacobian(h_function, states[:vel_idx], *camera_params[j])
    
    # Measurement Covariance R
    bad_point_mask = np.repeat(likelihood<dlc_thresh, 2)
    dlc_cov_arr = dlc_cov*np.ones((n_cams*n_markers*2))
    dlc_cov_arr[bad_point_mask] = max_pixel_err # change this to be independent of cam res?
    R = np.diag(dlc_cov_arr**2)

    # Residual
    residual = z_k - h

    # Residual Covariance S
    S = (H @ P @ H.T) + R
    temp = sigma_bound*np.sqrt(np.diag(S)) # if measurement residual is worse than 3 sigma, set residual to 0 and rely on predicted state only
    for j in range(0, len(residual), 2):
        if np.abs(residual[j])>temp[j] or np.abs(residual[j+1])>temp[j+1]:
            residual[j:j+2] = 0
            outliers_ignored += 1
        
    # Kalman Gain
    K = P @ H.T @ np.linalg.inv(S)

    # Correction
    states = states + K @ residual
    states_est_hist[i] = states

    # Update State Covariance
    P = (np.eye(K.shape[0]) - K @ H) @ P
    P_est_hist[i] = P
    
print("Outliers ignored:", outliers_ignored)

# Run Kalman Smoother
smooth_states_est_hist = states_est_hist.copy()
smooth_P_est_hist = P_est_hist.copy()
for i in range(n_frames-2, 0, -1):
    A = P_est_hist[i] @ F.T @ np.linalg.inv(P_pred_hist[i+1])
    smooth_states_est_hist[i] = states_est_hist[i] + A @ (smooth_states_est_hist[i+1] - states_pred_hist[i+1])
    smooth_P_est_hist[i] = P_est_hist[i] + A @ (smooth_P_est_hist[i+1] - P_pred_hist[i+1]) @ A.T
    
print("\nKalman Smoother complete!")

# Save EKF results

In [ ]:
save_ekf(states_est_hist, smooth_states_est_hist, DATA_DIR, start_frame, include_lure, dlc_thresh)

In [ ]:

fig, axs = plt.subplots(8, 2, figsize=(13,30))
# Head
axs[0,0].plot(states_est_hist[:, [idx['x_0'], idx['y_0'], idx['z_0']]])
axs[0,0].plot(smooth_states_est_hist[:, [idx['x_0'], idx['y_0'], idx['z_0']]], linestyle='dashed')
axs[0,0].set_title("Head positions")
axs[0,0].legend(['x 0', 'y 0', 'z 0', 'x 0 (smoothed)', 'y 0 (smoothed)', 'z 0 (smoothed)'])

# Head
# axs[0,1].plot(states_est_hist[:, [d_x_0, d_y_0, d_z_0]])
# axs[0,1].plot(smooth_states_est_hist[:, [d_x_0, d_y_0, d_z_0]], linestyle='dashed')
# axs[0,1].set_title("Head velocity")
# axs[0,1].legend(['dx 0', 'dy 0', 'dz 0', 'dx 0 (smoothed)', 'dy 0 (smoothed)', 'dz 0 (smoothed)'])

axs[1,0].plot(states_est_hist[:, [idx['phi_0'], idx['theta_0'], idx['psi_0']]])
axs[1,0].plot(smooth_states_est_hist[:, [idx['phi_0'], idx['theta_0'], idx['psi_0']]], linestyle='dashed')
axs[1,0].set_title("Head angles")
axs[1,0].legend(['phi 0', 'theta 0', 'psi 0', 'phi 0 (smoothed)', 'theta 0 (smoothed)', 'psi 0 (smoothed)'])

axs[1,1].plot(states_est_hist[:, [phi_1, theta_1, psi_1]])
axs[1,1].plot(smooth_states_est_hist[:, [phi_1, theta_1, psi_1]], linestyle='dashed')
axs[1,1].set_title("Neck angles")
axs[1,1].legend(['phi 1', 'theta 1', 'psi 1', 'phi 1 (smoothed)', 'theta 1 (smoothed)', 'psi 1 (smoothed)'])

axs[2,0].plot(states_est_hist[:, [theta_2]])
axs[2,0].plot(smooth_states_est_hist[:, [theta_2]], linestyle='dashed')
axs[2,0].set_title("Front torso angles")
axs[2,0].legend(['theta_2', 'theta_2 (smoothed)'])

axs[2,1].plot(states_est_hist[:, [phi_3, theta_3, psi_3]])
axs[2,1].plot(smooth_states_est_hist[:, [phi_3, theta_3, psi_3]], linestyle='dashed')
axs[2,1].set_title("Back torso angles")
axs[2,1].legend(['phi_3', 'theta_3', 'psi_3', 'phi_3 (smoothed)', 'theta_3 (smoothed)', 'psi_3 (smoothed)'])

axs[3,0].plot(states_est_hist[:, [theta_4, psi_4]])
axs[3,0].plot(smooth_states_est_hist[:, [theta_4, psi_4]], linestyle='dashed')
axs[3,0].set_title("Tail base")
axs[3,0].legend(['theta_4', 'psi_4','theta_4 (smoothed)', 'psi_4 (smoothed)'])

axs[3,1].plot(states_est_hist[:, [theta_5, psi_5]])
axs[3,1].plot(smooth_states_est_hist[:, [theta_5, psi_5]], linestyle='dashed')
axs[3,1].set_title("Tail Mid")
axs[3,1].legend(['theta_5', 'psi_5', 'theta_5 (smoothed)', 'psi_5 (smoothed)'])

axs[4,0].plot(states_est_hist[:, [theta_6]])
axs[4,0].plot(smooth_states_est_hist[:, [theta_6]], linestyle='dashed')
axs[4,0].set_title("Left shoulder angles")
axs[4,0].legend(['theta_6', 'theta_6 (smoothed)'])

axs[4,1].plot(states_est_hist[:, [theta_7]])
axs[4,1].plot(smooth_states_est_hist[:, [theta_7]], linestyle='dashed')
axs[4,1].set_title("Left front knee angle")
axs[4,1].legend(['theta_7', 'theta_7 (smoothed)'])

axs[5,0].plot(states_est_hist[:, [theta_8]])
axs[5,0].plot(smooth_states_est_hist[:, [theta_8]], linestyle='dashed')
axs[5,0].set_title("Right shoulder angles")
axs[5,0].legend(['theta_8', 'theta_8 (smoothed)'])

axs[5,1].plot(states_est_hist[:, [theta_9]])
axs[5,1].plot(smooth_states_est_hist[:, [theta_9]], linestyle='dashed')
axs[5,1].set_title("Right front knee angle")
axs[5,1].legend(['theta_9', 'theta_9 (smoothed)'])

axs[6,0].plot(states_est_hist[:, [theta_10]])
axs[6,0].plot(smooth_states_est_hist[:, [theta_10]], linestyle='dashed')
axs[6,0].set_title("Left hip angle")
axs[6,0].legend(['theta_10', 'theta_10 (smoothed)'])

axs[6,1].plot(states_est_hist[:, [theta_11]])
axs[6,1].plot(smooth_states_est_hist[:, [theta_11]], linestyle='dashed')
axs[6,1].set_title("Left back knee angle")
axs[6,1].legend(['theta_11', 'theta_11 (smoothed)'])

axs[7,0].plot(states_est_hist[:, [theta_12]])
axs[7,0].plot(smooth_states_est_hist[:, [theta_12]], linestyle='dashed')
axs[7,0].set_title("Right hip angle")
axs[7,0].legend(['theta_12', 'theta_12 (smoothed)'])

axs[7,1].plot(states_est_hist[:, [theta_13]])
axs[7,1].plot(smooth_states_est_hist[:, [theta_13]], linestyle='dashed')
axs[7,1].set_title("Right back knee angle")
axs[7,1].legend(['theta_13', 'theta_13 (smoothed)'])

In [ ]:
with open(os.path.join(DATA_DIR, "ekf", 'ekf_with_lure.pickle'), 'rb') as f:
    ekf_data = pickle.load(f)
    
positions = ekf_data["smooth_positions"]
# a = PoseAnimation(positions)
# a.animation()
ca = plotting.Cheetah(positions, DATA_DIR,dark_mode=True)
ca.animation()